In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import itertools
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import re
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
tfidf_df = pd.read_pickle("TfIdfSimilarity.pkl")

In [3]:
tfidf_df.head()

,indices
0,"[0, 10906, 157601, 536491, 501940, 82165, 2036..."
1,"[1, 427218, 511076, 24525, 577051, 96327, 2624..."
2,"[2, 61163, 592065, 318913, 333359, 486290, 360..."
3,"[3, 77662, 116585, 184548, 50193, 600110, 2372..."
4,"[4, 168393, 113970, 106661, 110259, 23364, 186..."


In [4]:
questions_lemmatized = pd.read_csv('LemmatizedQuestions.csv', encoding="ISO-8859-1")

In [7]:
pd.set_option('display.max_colwidth', -1)

In [10]:
questions_lemmatized.loc[22, "QuestionTitle"]

'Convert Bytes to Floating Point Numbers in Python'

In [11]:
for i in [22, 7, 33]:
    indices = tfidf_df.indices[i]
    sim_qs = questions_lemmatized.loc[indices, "QuestionTitle"]
    print(i)
    with pd.option_context('display.max_rows', None, 'display.max_columns', 3):
        print(sim_qs)

22
22        Convert Bytes to Floating Point Numbers in Python                             
46114     How to convert floating point number to base 3 in python                      
211985    How to convert floating point number in python?                               
279200    Indexing array in python results in lost scientific notation                  
57677     Reading 32 bit signed ieee 754 floating points from a binary file with python?
156849    Convert floating point number to certain precision, then copy to String       
16819     Correct way to emulate single precision floating point in python?             
221606    How to print floating point numbers as it is without any truncation in python?
197881    How can I convert a floating point number to an integer in python?            
222833    Extract first N position of floating number                                   
Name: QuestionTitle, dtype: object
7
7         How do I use Python's itertools.groupby()?                  

# RUN THESE CELLS PART 2

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import itertools
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import re
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
questions_lemmatized = pd.read_csv('LemmatizedQuestions.csv', encoding="ISO-8859-1")

In [ ]:
# Non-code text
TagFreeNonCodeTextLemmatized = questions_lemmatized.TagFreeNonCodeTextLemmatized.tolist()
TagFreeNonCodeTextLemmatized = [[w[1:-1] for w in q[1:-1].split(", ")] for q in TagFreeNonCodeTextLemmatized]

In [ ]:
# Code text
CodeTextLemmatized = questions_lemmatized.CodeTextLemmatized.tolist()
CodeTextLemmatized = [[w[1:-1] for w in q[1:-1].split(", ")] for q in CodeTextLemmatized]

In [ ]:
# Combined non-code and code
AllTextLemmatized = []
for i in range(len(TagFreeNonCodeTextLemmatized)):
    noncode = TagFreeNonCodeTextLemmatized[i][:]
    for w in CodeTextLemmatized[i]:
        if w != '':
            noncode.append(w)
    AllTextLemmatized.append(" ".join(noncode))

In [ ]:
# tfidf
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=10000) # limit vocabulary size to 10,000
tfidf_question = tfidf_vectorizer.fit_transform(AllTextLemmatized)

In [ ]:
# compute cosine similarities of questions
tfidf_question_cosine_similarities = [] # list of indices of top 10 most cosine similar
for i in range(100):
    similarity_indices = cosine_similarity(tfidf_question[i], tfidf_question).flatten()
    tfidf_question_cosine_similarities.append(similarity_indices.argsort()[:-11:-1])

In [ ]:
# save to df
tfidf_df = pd.DataFrame((_ for _ in itertools.zip_longest(tfidf_question_cosine_similarities)), columns=['indices'])
tfidf_df.to_pickle("TfIdfSimilarity.pkl")

# RUN THESE CELLS

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import itertools
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import re
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
noncode_embedding_df = pd.read_pickle("NoncodeTextWordEmbeddings")
code_embedding_df = pd.read_pickle("CodeTextWordEmbeddings")

In [ ]:
combined_embedding = []
for i in range(noncode_embedding_df.shape[0]):
    q_embedding = np.append(noncode_embedding_df.iloc[i,:][0], code_embedding_df.iloc[i,:][0])
    combined_embedding.append(q_embedding)

In [ ]:
combined_embedding_df = pd.DataFrame((_ for _ in itertools.zip_longest(combined_embedding)), columns=['indices'])
combined_embedding_df.to_pickle("CombinedWordEmbeddings.pkl")

In [ ]:
import warnings; warnings.simplefilter('ignore')

In [ ]:
# list of indices of top 10 most cosine similar for each question
word2vec_question_cosine_similarities = []
for i in range(len(combined_embedding)):
    similarity_indices = cosine_similarity(combined_embedding[i], combined_embedding).flatten()
    word2vec_question_cosine_similarities.append(similarity_indices.argsort()[:-11:-1])

In [ ]:
# save to df
word2vec_similarity_df = pd.DataFrame((_ for _ in itertools.zip_longest(word2vec_question_cosine_similarities)), columns=['indices'])
word2vec_similarity_df.to_pickle("word2vecSimilarity.pkl")

In [2]:
word2vec_similarity_df = pd.read_pickle("word2vecSimilarity.pkl")

In [3]:
word2vec_similarity_df.head()

,indices
0,"[0, 18281, 425603, 484729, 63804, 215319, 3383..."
1,"[1, 8749, 158621, 584677, 167832, 344385, 1266..."
2,"[2, 203500, 167128, 11119, 58898, 95965, 15147..."
3,"[3, 178120, 450349, 421067, 499326, 518685, 13..."
4,"[4, 394072, 31598, 445247, 538159, 642, 415426..."


In [6]:
word2vec_similarity_df.indices[0]

array([     0,  18281, 425603, 484729,  63804, 215319, 338349, 155286,
        87413, 122776], dtype=int64)

In [4]:
questions_lemmatized = pd.read_csv('LemmatizedQuestions.csv', encoding="ISO-8859-1")

In [5]:
questions_lemmatized.head()

,QID,QuestionUserId,QuestionCreateDate,QuestionScore,QuestionTitle,QuestionBody,NumAnswers,QuestionTitleAndBody,CodeText,TagFreeNonCodeText,CodeTextLemmatized,TagFreeNonCodeTextLemmatized
0,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...,4.0,How can I find the full path to a font from it...,NaN,How can I find the full path to a font from it...,[],"['how', 'can', 'i', 'find', 'the', 'full', 'pa..."
1,502,147.0,2008-08-02T17:01:58Z,27,Get a preview JPEG of a PDF on Windows?,<p>I have a cross-platform (Python) applicatio...,3.0,Get a preview JPEG of a PDF on Windows? <p>I h...,NaN,Get a preview JPEG of a PDF on Windows? I have...,[],"['get', 'a', 'preview', 'jpeg', 'of', 'a', 'pd..."
2,535,154.0,2008-08-02T18:43:54Z,40,Continuous Integration System for a Python Cod...,<p>I'm starting work on a hobby project with a...,7.0,Continuous Integration System for a Python Cod...,NaN,Continuous Integration System for a Python Cod...,[],"['continuous', 'integration', 'system', 'for',..."
3,594,116.0,2008-08-03T01:15:08Z,25,cx_Oracle: How do I iterate over a result set?,<p>There are several ways to iterate over a re...,3.0,cx_Oracle: How do I iterate over a result set?...,NaN,cx_Oracle: How do I iterate over a result set?...,[],"['cx_oracle', 'how', 'do', 'i', 'iterate', 'ov..."
4,683,199.0,2008-08-03T13:19:16Z,28,Using 'in' to match an attribute of Python obj...,<p>I don't remember whether I was dreaming or ...,8.0,Using 'in' to match an attribute of Python obj...,"foo in iter_attr(array of python objects, attr...",Using 'in' to match an attribute of Python obj...,"['foo', 'in', 'iter_attrarray', 'of', 'python'...","['using', 'in', 'to', 'match', 'an', 'attribut..."


In [16]:
for i in range(20):
    indices = word2vec_similarity_df.indices[i][1:]
    sim_qs = questions_lemmatized.loc[indices, "QuestionTitle"]
    print(i)
    with pd.option_context('display.max_rows', None, 'display.max_columns', 3):
        print(sim_qs)

0
18281     Generate image for each font on a linux system using Python              
425603    Using QPython Console with MIT App Inventor                              
484729    Using DeviceCapabilities with pywin32                                    
63804     How can I write color text to a textbox using python and the win32 api's?
215319    get vim to search the python path                                        
338349    Where can I locate the library for python base functions in Linux?       
155286    Running Blender python script outside of blender                         
87413     Recognize images in Python                                               
122776    Generating an image-report with Python                                   
Name: QuestionTitle, dtype: object
1
8749      Display jpg images in python                                              
158621    Retrieve JPEG image from a redirected URL and display it on a GUI window  
584677    Get window xid from handl

In [17]:
for i in range(20, 40):
    indices = word2vec_similarity_df.indices[i][1:]
    sim_qs = questions_lemmatized.loc[indices, "QuestionTitle"]
    print(i)
    with pd.option_context('display.max_rows', None, 'display.max_columns', 3):
        print(sim_qs)

20
136287    Application that can load both C/C++ and Python plugins                                     
78997     Recommend a library for testing the 'closedness' of '.stl' geometries?                      
316134    Looking into getting into Python specifically for a project that involves threads and graphs
131973    Render 3D model orthographically in Python                                                  
18943     Is it possible to use re2 from Python?                                                      
464631    Fortran or C and f2py to solve differential equations                                       
63066     Python interface in C++ Windows Form Application                                            
562748    Best way to package Python app within an autotools project                                  
4039      Multiprocessing or Multithreading?                                                          
Name: QuestionTitle, dtype: object
21
31213     Language/GUI library t

In [10]:
pd.set_option('display.max_colwidth', -1)

### Methods

Transforming text to vectors:
- Use tfidf
- Use word2vec:
    - Train non-code on 50D
    - Train code on 50D
    - Concatenate vectors



After getting vector representation, use similarity metrics to find similar questions. Also hopefully use some clustering method is get cluster features for the feature engineering part.

Similarity metrics:
- top 10 most cosine similar

Clustering method:
- Maybe use DBscan
- PCA or t-SNE
- *The clustering method could possible indicate interesting sub-question types, e.g. one cluster is for non-code questions, another is for debugging, another is for conceptual.*

### Vector Representations

tf-idf

In [ ]:
questions_lemmatized["TagFreeNonCodeTextLemmatized"][0]

In [ ]:
# Non-code text
TagFreeNonCodeTextLemmatized = questions_lemmatized.TagFreeNonCodeTextLemmatized.tolist()
TagFreeNonCodeTextLemmatized = [[w[1:-1] for w in q[1:-1].split(", ")] for q in TagFreeNonCodeTextLemmatized]

In [ ]:
# Code text
CodeTextLemmatized = questions_lemmatized.CodeTextLemmatized.tolist()
CodeTextLemmatized = [[w[1:-1] for w in q[1:-1].split(", ")] for q in CodeTextLemmatized]

In [ ]:
# Combined non-code and code
AllTextLemmatized = []
for i in range(len(TagFreeNonCodeTextLemmatized)):
    noncode = TagFreeNonCodeTextLemmatized[i][:]
    for w in CodeTextLemmatized[i]:
        if w != '':
            noncode.append(w)
    AllTextLemmatized.append(" ".join(noncode))

In [ ]:
AllTextLemmatized[0]

In [ ]:
# tfidf
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=10000) # limit vocabulary size to 10,000
tfidf_question = tfidf_vectorizer.fit_transform(AllTextLemmatized)

In [ ]:
tfidf_question.shape

In [ ]:
# compute cosine similarities of questions
tfidf_question_cosine_similarities = [] # list of indices of top 10 most cosine similar
for i in range(len(questions_lemmatized.index)):
    similarity_indices = cosine_similarity(tfidf_question[i], tfidf_question).flatten()
    tfidf_question_cosine_similarities.append(similarity_indices.argsort()[:-11:-1])

In [ ]:
tfidf_question_cosine_similarities[0]

In [ ]:
# save to df
tfidf_df = pd.DataFrame((_ for _ in itertools.zip_longest(tfidf_question_cosine_similarities)), columns=['indices'])
tfidf_df.to_pickle("TfIdfSimilarity.pkl")

In [ ]:
tdfidf_df = pd.read_pickle("TfIdfSimilarity")

In [ ]:
tdfidf_df.head()

**word2vec for non-code**

In [ ]:
import gensim
import nltk
from nltk import word_tokenize
from nltk import tokenize
from nltk.data import find

In [ ]:
# word2vec model for non-code text
noncode_model = gensim.models.Word2Vec(TagFreeNonCodeTextLemmatized, min_count=10, size=50, window=5)

In [ ]:
len(noncode_model.wv.vocab)

In [ ]:
noncode_unique_words = {}
for q in TagFreeNonCodeTextLemmatized:
    for w in q:
        if not noncode_unique_words.get(w):
            noncode_unique_words[w] = 0
        noncode_unique_words[w] += 1

In [ ]:
words = list(noncode_unique_words.keys())
len(words)

*Get word vectors*

In [ ]:
# Word vectors
noncode_vector_list=[] ## n by d matrix containing words and their respective vectors
for word, cnt in noncode_unique_words.items():
    if cnt >= 10:
        noncode_vector_list.append(noncode_model[word])

In [ ]:
len(noncode_vector_list)

In [ ]:
# save model
noncode_model.save("noncode_word2vec.bin")

In [ ]:
# load model
noncode_model = gensim.models.Word2Vec.load("noncode_word2vec.bin")

*Sum up word vectors*

In [ ]:
noncode_q_embedding = []
for q in TagFreeNonCodeTextLemmatized:
    q_embedding = np.zeros(50)
    for word in q:
        if noncode_unique_words[word] > 10:
            q_embedding += noncode_model[word]
    noncode_q_embedding.append(q_embedding)

In [ ]:
len(noncode_q_embedding)

In [ ]:
noncode_q_embedding[0]

In [ ]:
noncode_embedding_df = pd.DataFrame((_ for _ in itertools.zip_longest(noncode_q_embedding)), columns=['embedding'])

In [ ]:
noncode_embedding_df.head()

In [ ]:
# save df
noncode_embedding_df.to_pickle("NoncodeTextWordEmbeddings.pkl")

**word2vec for code text**

In [ ]:
# word2vec model for code text
code_model = gensim.models.Word2Vec(CodeTextLemmatized, min_count=10, size=50, window=5)

In [ ]:
len(code_model.wv.vocab)

In [ ]:
# get set of unique words and counts
code_unique_words = {}
for q in CodeTextLemmatized:
    for w in q:
        if not code_unique_words.get(w):
            code_unique_words[w] = 0
        code_unique_words[w] += 1

In [ ]:
# save model
code_model.save("code_word2vec.bin")

In [ ]:
# load model
code_model = gensim.models.Word2Vec.load("code_word2vec.bin")

In [ ]:
# sum up word vectors
code_q_embedding = []
for q in CodeTextLemmatized:
    q_embedding = np.zeros(50)
    for word in q:
        if code_unique_words[word] > 10:
            q_embedding += code_model[word]
    code_q_embedding.append(q_embedding)

In [ ]:
len(code_q_embedding)

In [ ]:
code_embedding_df = pd.DataFrame((_ for _ in itertools.zip_longest(code_q_embedding)), columns=['embedding'])

In [ ]:
code_embedding_df.head()

In [ ]:
# save df
code_embedding_df.to_pickle("CodeTextWordEmbeddings.pkl")

**Get top 10 most cosine similar questions**

*Concatenate code and non-code embeddings*

In [ ]:
code_embedding_df = pd.read_pickle("CodeTextWordEmbeddings")

In [ ]:
noncode_embedding_df = pd.read_pickle("NoncodeTextWordEmbeddings")

In [ ]:
noncode_embedding_df.iloc[0,:][0]#.as_matrix()[0]

In [ ]:
combined_embedding = []
for i in range(5):# range(noncode_embedding_df.shape[0]):
    q_embedding = np.append(noncode_embedding_df.iloc[i,:][0], code_embedding_df.iloc[i,:][0])
    combined_embedding.append(q_embedding)

*Get top 10 most cosine similar word embeddings*

In [ ]:
combined_embedding

In [ ]:
# list of indices of top 10 most cosine similar for each question
word2vec_question_cosine_similarities = []
for i in range(len(combined_embedding)):
    similarity_indices = cosine_similarity(combined_embedding[i], combined_embedding).flatten()
    word2vec_question_cosine_similarities.append(similarity_indices.argsort()[:-11:-1])

In [ ]:
word2vec_question_cosine_similarities

In [ ]:
# save to df
word2vec_similarity_df = pd.DataFrame((_ for _ in itertools.zip_longest(word2vec_question_cosine_similarities)), columns=['indices'])
word2vec_similarity_df.to_pickle("word2vecSimilarity.pkl")

In [ ]:
word2vec_similarity_df.head()